# Sampling on a real quantum computer

In the previous section ([Sampling simulation](sampling_simulation.ipynb)), we described how to estimate expectation value of operators using sampling measurements on a quantum circuit simulator. Since QURI Parts is designed to be platform independent, you can execute almost the same code on a real quantum computer.

## Prerequisite

This section requires topics described in the previous section ([Sampling simulation](sampling_simulation.ipynb)), so you need to read it before this section.

We use [Amazon Braket](https://aws.amazon.com/braket/) as an example of a platform with real quantum computers. In order to use Braket devices provided on AWS, you need to have an AWS account and enable Braket service. Please see [Amazon Braket Documentation](https://docs.aws.amazon.com/braket/index.html) for details. In this section, instead, we use the local simulator included in [Amazon Braket SDK](https://amazon-braket-sdk-python.readthedocs.io/en/latest/index.html), which does not require an AWS account. The Braket devices provided on AWS and the local simulator have the same interface, you can simply replace them each other.

QURI Parts modules used in this tutorial: `quri-parts-circuit`, `quri-parts-core` and `quri-parts-braket`. You can install them as follows:

In [ ]:
!pip install "quri-parts[braket]"

## Prepare a circuit

As a preparation, we create a circuit to be sampled:

In [1]:
from math import pi
from quri_parts.circuit import QuantumCircuit
# A circuit with 4 qubits
circuit = QuantumCircuit(4)
circuit.add_X_gate(0)
circuit.add_H_gate(1)
circuit.add_Y_gate(2)
circuit.add_CNOT_gate(1, 2)
circuit.add_RX_gate(3, pi/4)

## SamplingBackend and Sampler

In order to use a real device, you need to create a `SamplingBackend` object and then a `Sampler` using the backend. The `SamplingBackend` provides a unified interface for handling various backend devices, computation jobs for the devices and results of the jobs. 

How to create a `SamplingBackend` object depends on the used backend. For Braket devices, you can create a `BraketSamplingBackend` by passing a `braket.devices.Device` object (provided by Amazon Braket SDK):

In [2]:
from braket.aws import AwsDevice
from braket.devices import LocalSimulator

# A device for QPU provided on AWS
# device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-2")

# A device for the local simulator
device = LocalSimulator()

In [3]:
from quri_parts.braket.backend import BraketSamplingBackend

# Create a SamplingBackend with the device
backend = BraketSamplingBackend(device)

It is possible to use this backend directly, though it is usually unnecessary as we will see below. The `SamplingBackend` has `sample()` method, which returns a `SamplingJob` object, and you can extract a result of the sampling job:

In [4]:
job = backend.sample(circuit, n_shots=1000)
result = job.result()
print(result.counts)

Counter({5: 445, 3: 397, 13: 82, 11: 76})


Instead of using the backend directly, you can create a `Sampler` from it.

In [5]:
from quri_parts.core.sampling import create_sampler_from_sampling_backend

sampler = create_sampler_from_sampling_backend(backend)
sampling_result = sampler(circuit, 1000)
print(sampling_result)

# A concurrent sampler can also be created
from quri_parts.core.sampling import create_concurrent_sampler_from_sampling_backend

concurrent_sampler = create_concurrent_sampler_from_sampling_backend(backend)

Counter({3: 448, 5: 417, 13: 70, 11: 65})


With the `Sampler`, you can perform sampling estimation of an operator on a quantum state exactly in the same way as described in [the previous section](sampling_simulation.ipynb):

In [6]:
from quri_parts.core.operator import Operator, pauli_label, PAULI_IDENTITY
op = Operator({
    pauli_label("Z0"): 0.25,
    pauli_label("Z1 Z2"): 2.0,
    pauli_label("X1 X2"): 0.5 + 0.25j,
    pauli_label("Z1 Y3"): 1.0j,
    pauli_label("Z2 Y3"): 1.5 + 0.5j,
    pauli_label("X1 Y3"): 2.0j,
    PAULI_IDENTITY: 3.0,
})

from quri_parts.core.state import ComputationalBasisState
initial_state = ComputationalBasisState(4, bits=0b0101)

from quri_parts.core.measurement import bitwise_commuting_pauli_measurement
from quri_parts.core.sampling.shots_allocator import create_weighted_random_shots_allocator
allocator = create_weighted_random_shots_allocator(seed=777)

from quri_parts.core.estimator.sampling import sampling_estimate
estimate = sampling_estimate(
    op,            # Operator to estimate
    initial_state, # Initial (circuit) state
    5000,          # Total sampling shots
    concurrent_sampler, # ConcurrentSampler
    bitwise_commuting_pauli_measurement, # Factory function for CommutablePauliSetMeasurement
    allocator,     # PauliSamplingShotsAllocator
)
print(f"Estimated expectation value: {estimate.value}")
print(f"Standard error of estimation: {estimate.error}")

Estimated expectation value: (0.7253113103278891-0.007043979821499793j)
Standard error of estimation: 0.07072986943220279


You can also create a `QuantumEstimator` that performs sampling estimation:

In [7]:
from quri_parts.core.estimator.sampling import create_sampling_estimator
estimator = create_sampling_estimator(
    5000,          # Total sampling shots
    concurrent_sampler, # ConcurrentSampler
    bitwise_commuting_pauli_measurement, # Factory function for CommutablePauliSetMeasurement
    allocator,     # PauliSamplingShotsAllocator
)
estimate = estimator(op, initial_state)
print(f"Estimated expectation value: {estimate.value}")
print(f"Standard error of estimation: {estimate.error}")

Estimated expectation value: (0.6310222899695139-0.04464326805968584j)
Standard error of estimation: 0.07028525906672647


## Qubit mapping

When you use a real quantum device, you may want to use specific device qubits selected by inspecting calibration data of the device. A `SamplingBackend` supports such usage with `qubit_mapping` argument. With `qubit_mapping` you can specify an arbitrary one-to-one mapping between qubit indices in the input circuit and device qubits. For example, if you want to map qubits in the circuit into device qubits as 0 → 3, 1 → 2, 2 → 0 and 3 → 1, you can specify the mapping as follows:

In [8]:
backend = BraketSamplingBackend(device, qubit_mapping={0: 3, 1: 2, 2: 0, 3: 1})
sampler = create_sampler_from_sampling_backend(backend)
sampling_result = sampler(circuit, 1000)
print(sampling_result)

{3: 442, 5: 400, 13: 70, 11: 88}


The result looks similar to one with no qubit mapping, since the measurement result from the device is mapped backward so that it is interpreted in terms of the original qubit indices.

<div class="alert alert-info">
    You may notice that the above mapping is a permutation of the original qubit indices and device qubits with indices larger than 3 are not involved. The reason for choosing such a mapping is to avoid an error of <code>LocalSimulator</code>: the <code>LocalSimulator</code> does not accept non-contiguous qubit indices. On the other hand, the qubit mapping feature of the <code>SamplingBackend</code> accepts such a mapping, as shown below.
</div>

When you apply qubit mapping to devices provided on AWS, you will need to [enable manual qubit allocation by passing disable_qubit_rewiring=True](https://docs.aws.amazon.com/braket/latest/developerguide/braket-constructing-circuit.html#manual-qubit-allocation) to the device. You can specify such an argument (i.e. keyword arguments for `run` method of a `braket.devices.Device` object) via `run_kwargs` argument of the `BraketSamplingBackend` object:

In [9]:
# Commented out because it requires an access to a real device on AWS

# device = AwsDevice("arn:aws:braket:us-west-1::device/qpu/rigetti/Aspen-M-2")
# backend = BraketSamplingBackend(
#     device,
#     qubit_mapping={0: 10, 1: 13, 2: 17, 3: 21},
#     run_kwargs={"disable_qubit_rewiring": True},
# )
# sampler = create_sampler_from_sampling_backend(backend)
# sampling_result = sampler(circuit, 1000)
# print(sampling_result)

## Circuit transpilation before execution

When the `SamplingBackend` receives an input circuit, it performs circuit transpilation before sending the circuit to its backend since each device can have a different supported gate set. The transpilation performed by default depends on the backend; in the case of `BraketSamplingBackend`, it uses `quri_parts.braket.circuit.BraketTranspiler` for all devices, and also performs some device-specific transpilation defined in `quri_parts.braket.backend.transpiler`. It is possible to change the former one (device-independent transpilation) by supplying `circuit_transpiler` argument to `BraketSamplingBackend`.